In [4]:
import scipy.io
import os
import shutil
try:
    from urllib import urlretrieve # python2
except:
    from urllib.request import urlretrieve # python3
#import CSA

In [5]:
import data as data
import numpy as np
import matplotlib.pyplot as plt
import torch as T
import torch
device = T.device("cuda")  # apply to Tensor or Module
import time
from tqdm import tqdm
from train_objectives import SAD, SID

In [6]:
torch.cuda.empty_cache()

In [7]:
mat = scipy.io.loadmat( 'PaviaU.mat' )
img = mat[ 'paviaU' ]

# create a hyperspectral dataset object from the numpy array
hypData = data.HypImg( img )

# pre-process data to make the model easier to train
hypData.pre_process( 'minmax' )

In [8]:
# create data iterator objects for training and validation using the pre-processed data
trainSamples = 200000
valSamples = 100
dataTrain = data.Iterator( dataSamples=hypData.spectraPrep[:trainSamples, :],
                        targets=hypData.spectraPrep[:trainSamples, :], batchSize=4 )
dataVal = data.Iterator( dataSamples=hypData.spectraPrep[trainSamples:trainSamples+valSamples, :],
                        targets=hypData.spectraPrep[trainSamples:trainSamples+valSamples, :] )


In [9]:
# shuffle training data
dataTrain.shuffle()

In [10]:
train_data = torch.tensor(dataTrain.dataSamples.astype(np.float32))
train_data

tensor([[4.4879e-01, 8.8827e-01, 7.2253e-01,  ..., 3.9665e-01, 4.0969e-01,
         3.9293e-01],
        [7.7602e-02, 6.6937e-02, 2.6848e-02,  ..., 8.4112e-01, 8.4737e-01,
         8.6061e-01],
        [1.0185e-01, 3.4294e-07, 5.5213e-02,  ..., 6.6907e-01, 6.7901e-01,
         6.7901e-01],
        ...,
        [2.3142e-01, 1.9193e-01, 1.2961e-01,  ..., 9.4558e-01, 9.6665e-01,
         9.8215e-01],
        [1.3468e-01, 4.1493e-02, 3.8016e-02,  ..., 8.8688e-01, 9.0311e-01,
         9.1029e-01],
        [1.1197e-01, 8.2849e-02, 6.4725e-07,  ..., 2.8803e-01, 2.8479e-01,
         2.8414e-01]])

In [11]:
hypData.numBands
encoderSize=[50,30,10]
encoderSize=[hypData.numBands]+encoderSize
decodersize=encoderSize[::-1]
encoderSize,decodersize

([103, 50, 30, 10], [10, 30, 50, 103])

In [12]:
def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))

In [13]:
# -----------------------------------------------------------

class Net(T.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.enn1 = T.nn.Linear(103, 50)  # 64-16-2-16-64
    self.enn2 = T.nn.Linear(50, 30)
    self.enn3 = T.nn.Linear(30,10)
    self.dnn3 = T.nn.Linear(10, 30)  # 64-16-2-16-64
    self.dnn2 = T.nn.Linear(30, 50)
    self.dnn1 = T.nn.Linear(50,103)

  def encoder(self,x):
    z= T.relu(self.enn1(x))
    z= T.relu(self.enn2(z))
    z= T.relu(self.enn3(z))
    return z
  
  def decoder(self,x):
    z= T.relu(self.dnn3(x))
    z= T.relu(self.dnn2(z))
    z= self.dnn1(z)
    return z


  def forward(self, x):
    encoded=self.encoder(x)
    decoded= self.decoder(encoded)
    
    return encoded,decoded
    
net=Net().to(device)


In [14]:

inpu=torch.rand(1,103).to(device)
outp=net(inpu)
outp[1].shape


torch.Size([1, 103])

In [15]:
hypData.numBands

103

In [19]:
# -----------------------------------------------------------

force_cudnn_initialization()
# 0. setup
print("\nBegin UCI digits auto-reduce-viz demo job ")
T.manual_seed(1)
np.random.seed(1)

bat_size = 8184
train_ldr = T.utils.data.DataLoader(train_data,
batch_size=bat_size, shuffle=True)
  # 2. create network
print("\nCreating 64-16-2-16-63 autoencoder ")
net = Net().to("cuda:0")

# 3. train model
max_epochs = 1000
ep_log_interval = 5
lrn_rate = 0.0001

loss_func1 = T.nn.MSELoss()
#loss_func = SID()
optimizer = T.optim.Adam(net.parameters(), lr=lrn_rate)

print("\nbat_size = %3d " % bat_size)
#print("loss = " + str(csa))
print("optimizer = Adam")
print("max_epochs = %3d " % max_epochs)
print("lrn_rate = %0.3f " % lrn_rate)

print("\nStarting training")
net = net.train()


for epoch in range(0, max_epochs):
  time.sleep(0.5)
  loop= tqdm(enumerate(train_ldr), total=len(train_ldr),leave=True)
  epoch_loss = 0  # for one full epoch
  mseloss=0

  iterator = iter(train_ldr)
  
  for (batch_idx, batch) in loop:
    Z = next(iterator)
    Z = Z.view(Z.size()[0], -1)
    Z = Z.cuda()
    
    enc_out, dec_out = net(Z.float())
    loss1 = csa1(dec_out, Z.float())  
    loss1 = torch.sum(loss1).float()
    optimizer.zero_grad()
    loss1.backward()
    optimizer.step()
    
    X = batch.to(device)  # no targets needed
    
    '''optimizer.zero_grad()
    oupt = net(X)
    loss_obj = loss_func(oupt[1], X)  # note: X not Y'''
    #loss_obj1=loss_func1(oupt[1],X)
    epoch_loss += loss1.item()  # accumulate
    #mseloss+=loss_obj1.item()
    #loss_obj.backward()
    #optimizer.step()
    
    loop.set_description(f"Epoch [{epoch}/{max_epochs}]")
    loop.set_postfix(loss=str(epoch_loss))

    

  #if epoch % ep_log_interval == 0:
    
    
    #print("epoch = %4d   loss = %0.4f" % (epoch, epoch_loss)) 
print("Done ")

# 4. plot digits using reduced form
print("\nCreating graph from encoded data ")
net = net.eval()






Begin UCI digits auto-reduce-viz demo job 

Creating 64-16-2-16-63 autoencoder 

bat_size = 8184 
optimizer = Adam
max_epochs = 1000 
lrn_rate = 0.000 

Starting training


Epoch [3/1000]:  12%|█████                                     | 3/25 [00:00<00:02,  9.11it/s, loss=-81.73572731018066]


KeyboardInterrupt: 

In [ ]:
p=net(Z.float())
p.detach()

In [ ]:

torch.save(net, 'model_csa.pth')

In [ ]:
torch.save(net.state_dict(), 'net_model_csa.pth')

In [ ]:
net.load_state_dict(torch.load('net_model_csa.pth'))

In [ ]:
trex=torch.tensor(hypData.spectraPrep.astype(np.float32))
trex

In [ ]:
dataZ=net.encoder(trex.to("cuda"))
dataZ

In [ ]:
dataY = net.decoder(dataZ)
dataY

In [ ]:
imgZ = np.reshape(dataZ.to("cpu").detach().numpy(), (hypData.numRows, hypData.numCols, -1))

In [ ]:
imgY = np.reshape(dataY.to("cpu").detach().numpy(), (hypData.numRows, hypData.numCols, -1))

In [ ]:
imgX = np.reshape(hypData.spectraPrep, (hypData.numRows, hypData.numCols, -1))

In [ ]:
# visualise latent image using 3 out of the 10 dimensions
colourImg = imgZ.copy()
colourImg = colourImg[ :,:,np.argsort(-np.std(np.std(colourImg, axis=0), axis=0))[:3] ]
colourImg /= np.max(np.max(colourImg, axis=0), axis=0)

In [ ]:
plt.imshow(colourImg)

In [ ]:
# save plot of latent vector of 'meadow' spectra
fig = plt.figure()
plt.plot(imgZ[576, 210, :])
plt.xlabel('latent dimension')
plt.ylabel('latent value')
plt.title('meadow spectra')

In [ ]:
# save plot comparing pre-processed 'meadow' spectra input with decoder reconstruction
fig = plt.figure()
ax = plt.subplot(111)
ax.plot(range(hypData.numBands),imgX[576, 210, :],label='pre-processed input')
ax.plot(range(hypData.numBands),imgY[576, 210, :],label='reconstruction')
plt.xlabel('band')
plt.ylabel('value')
plt.title('meadow spectra')
ax.legend()

In [ ]:
loss_func = SID()
inpt1=torch.rand(3,103)
inpt2=torch.rand(3,103)
outp=csa1(inpt1,inpt2)
outp

In [ ]:
input=Z
target=dec_out
eps=1e5

normalize_inp = (input/torch.sum(input, dim=0)) + eps
normalize_tar = (target/torch.sum(target, dim=0)) + eps
sid = torch.sum(normalize_inp * torch.log(normalize_inp / normalize_tar) + normalize_tar * torch.log(normalize_tar / normalize_inp))
sid

In [ ]:
decout = dec_out.detach()
decout.shape

In [18]:
def csa1(input, target):
    norm_r = torch.nn.functional.normalize(torch.transpose(input,0,0),dim=0)
    norm_t = torch.nn.functional.normalize(torch.transpose(target,0,0),dim=0)
    mult = torch.multiply(norm_r,norm_t).sum()
    return 1-mult.sum()

In [14]:
def csa(input, target):
    normalize_r = (input/torch.sum(input, dim=0)) 
    normalize_t = (target/torch.sum(target, dim=0))
    mult = torch.multiply(normalize_r,normalize_t).sum()
    return 1-mult.sum()

In [ ]:
normalize_r = (input/torch.sum(input, dim=0)) 
normalize_t = (target/torch.sum(target, dim=0))
mult = torch.multiply(normalize_r,normalize_t).sum()
1-mult

In [ ]:
csaloss = torch.acos(mult)
csaloss